In [1]:
from math import floor, ceil
import numpy as np

In [2]:
def roundoff(N, n):
    b = N
    c = floor(N)
 
    # Counting the no. of digits
    # to the left of decimal point
    # in the given no.
    i = 0;
    while(b >= 1):
        b = b / 10
        i = i + 1
 
    d = n - i
    b = N
    b = b * pow(10, d)
    e = b + 0.5
    if (float(e) == float(ceil(b))):
        f = (ceil(b))
        h = f - 2
        if (h % 2 != 0):
            e = e - 1
    j = floor(e)
    m = pow(10, d)
    j = j / m
    return j

In [3]:
def np_roundoff(numbers , significant_digits):
    i=0
    while(i < len(numbers)):
        numbers[i] = roundoff(numbers[i], significant_digits)
        i =i+1
    return numbers

In [4]:
def GENP(A, b):
    '''
    Gaussian elimination with no pivoting.
    % input: A is an n x n nonsingular matrix
    %        b is an n x 1 vector
    % output: x is the solution of Ax=b.
    % post-condition: A and b have been modified. 
    '''
    n =  len(A)
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between A & b.", b.size, n)
    for pivot_row in range(n-1):
        for row in range(pivot_row+1, n):
            multiplier = A[row][pivot_row]/A[pivot_row][pivot_row]
            multiplier = r(multiplier, sd) # roundoff
            #the only one in this column since the rest are zero
            A[row][pivot_row] = multiplier
            for col in range(pivot_row + 1, n):
                A[row][col] = A[row][col] - multiplier*A[pivot_row][col]
                A[row][col]=r(A[row][col],sd) # roundoff
                
            #Equation solution column
            b[row] = b[row] - multiplier*b[pivot_row]
            b[row] = r(b[row],sd) #roundoff
    x = np.zeros(n)
    k = n-1
    x[k] = b[k]/A[k,k]
    x[k] = r(x[k],sd)
    while k >= 0:
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
        x[k] = r(x[k],sd)
        k = k-1
    return x

In [5]:
def GEPP(A, b, doPricing = True):
    '''
    Gaussian elimination with partial pivoting.
    
    input: A is an n x n numpy matrix
           b is an n x 1 numpy array
    output: x is the solution of Ax=b 
            with the entries permuted in 
            accordance with the pivoting 
            done by the algorithm
    post-condition: A and b have been modified.
    '''
    n = len(A)
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between"+
                         "A & b.", b.size, n)
    # k represents the current pivot row. Since GE traverses the matrix in the 
    # upper right triangle, we also use k for indicating the k-th diagonal 
    # column index.
    
    # Elimination
    for k in range(n-1):
        if doPricing:
            # Pivot
            maxindex = abs(A[k:,k]).argmax() + k
            if A[maxindex, k] == 0:
                raise ValueError("Matrix is singular.")
            # Swap
            if maxindex != k:
                A[[k,maxindex]] = A[[maxindex, k]]
                b[[k,maxindex]] = b[[maxindex, k]]
        else:
            if A[k, k] == 0:
                raise ValueError("Pivot element is zero. Try setting doPricing to True.")
        #Eliminate
        for row in range(k+1, n):
            multiplier = A[row,k]/A[k,k]
            multiplier = r(multiplier,sd) # roundoff
            A[row, k:] = A[row, k:] - multiplier*A[k, k:]
            A[row,k:] = nr(A[row,k:],sd) # roundoff
            b[row] = b[row] - multiplier*b[k]
            b[row] = r(b[row],sd) # roundoff
    # Back Substitution
    x = np.zeros(n)
    for k in range(n-1, -1, -1):
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
        x[k] = r(x[k], sd) # roundoff
    return x



In [6]:
sds=[3,4,5,6]

eq_1 = np.random.uniform(low=5.0, high=10.3, size=(2,)).tolist()
eq_2 =  np.random.uniform(low=5.0, high=20.3, size=(2,)).tolist()
coeff = np.array([eq_1, eq_2])
rhs = np.random.uniform(low=5.0, high=20.3, size=(2,)).tolist()
rhs = np.array(rhs)
rhs = rhs.T

r = roundoff
nr = np_roundoff

In [7]:
print('Gauss elimination without pivoting ...')
print('coeffients - ', coeff)
print()
print('rhs -', rhs)
print()
for sig_digit in sds:
    sd = sig_digit
    res = GENP(np.copy(coeff), np.copy(rhs))
    print(f'significant_digit= {sd}, X1 = {res[0]}, X2= {res[1]}')

Gauss elimination without pivoting ...
coeffients -  [[ 9.72795489  5.38156111]
 [ 5.90881368 18.21670753]]

rhs - [ 9.09393917 14.28096406]

significant_digit= 3, X1 = 0.612, X2= 0.584
significant_digit= 4, X1 = 0.6108, X2= 0.5858
significant_digit= 5, X1 = 0.61074, X2= 0.58584
significant_digit= 6, X1 = 0.610729, X2= 0.585852


In [8]:
print('Gauss elimination with pivoting ...')
print('coeffients - ', coeff)
print()
print('rhs -', rhs)
print()
for sig_digit in sds:
    sd = sig_digit
    res = GEPP(np.copy(coeff), np.copy(rhs))
    print(f'significant_digit= {sd}, X1 = {res[0]}, X2= {res[1]}')

Gauss elimination with pivoting ...
coeffients -  [[ 9.72795489  5.38156111]
 [ 5.90881368 18.21670753]]

rhs - [ 9.09393917 14.28096406]

significant_digit= 3, X1 = 0.612, X2= 0.584
significant_digit= 4, X1 = 0.6108, X2= 0.5858
significant_digit= 5, X1 = 0.61074, X2= 0.58584
significant_digit= 6, X1 = 0.610729, X2= 0.585852
